import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [83]:
import pyspark
!pip install pandas
import pandas
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col,aggregate,round,when
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [84]:
spark=SparkSession.builder.appName("somename").getOrCreate()

read csv with inferschema

In [85]:
df=spark.read.csv("ds_salaries.csv",header=True,inferSchema=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [86]:
df=spark.read.csv("ds_salaries.csv",header=True,inferSchema=True)

write schema of scv on screen

In [87]:
%%time
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)

CPU times: user 1.19 ms, sys: 2.06 ms, total: 3.26 ms
Wall time: 2.03 ms


create schema of this scv

In [88]:
schema=StructType([
    StructField("id",IntegerType(),True),
    StructField("work_year",StringType(),True),
    StructField("experience_level",StringType(),True),
    StructField("employment_type",StringType(),True),
    StructField("job_title",StringType(),True),
    StructField("salary",IntegerType(),True),
    StructField("salary_currency",StringType(),True),
    StructField("salary_in_usd",IntegerType(),True),
    StructField("employee_residence",StringType(),True),
    StructField("company_location",StringType(),True),
    StructField("company_size",StringType(),True),
])



restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [89]:
%%time
ds_salaries=spark.read.csv("ds_salaries.csv",schema=schema)

CPU times: user 3.99 ms, sys: 1.31 ms, total: 5.3 ms
Wall time: 14.1 ms


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [90]:
%%time
ds_salaries.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)

CPU times: user 0 ns, sys: 2.8 ms, total: 2.8 ms
Wall time: 1.75 ms


now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [91]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

25/09/01 08:30:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///workspace/1_PySpark_Basics/ds_salaries.csv


print data in dataframe using display(df.toPandas())

In [92]:
new_df=df.toPandas()
new_df.head()

25/09/01 08:30:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///workspace/1_PySpark_Basics/ds_salaries.csv


,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


create df_job_title that consists from all job_titles without duplicates

In [93]:
df_job_titles=df.select("job_title").distinct()

print all rows from df_job_titles without truncating jobs

In [94]:
df_job_titles.show()

+--------------------+
|           job_title|
+--------------------+
|3D Computer Visio...|
|  Lead Data Engineer|
|Head of Machine L...|
|     Data Specialist|
| Data Analytics Lead|
|Machine Learning ...|
|   Lead Data Analyst|
|Data Engineering ...|
|Staff Data Scientist|
|       ETL Developer|
|Director of Data ...|
|Product Data Analyst|
|Principal Data Sc...|
|        AI Scientist|
|Director of Data ...|
|Machine Learning ...|
| Lead Data Scientist|
|Machine Learning ...|
|Data Science Engi...|
|Machine Learning ...|
+--------------------+
only showing top 20 rows


create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [95]:
df_analytic=df.groupBy("job_title").agg(min("salary_in_usd").alias("min_salary"),\
                                        max("salary_in_usd").alias("max_salary"),\
                                        round(avg("salary_in_usd"),1).alias("avg_salary"))

print all rows from df_analytic without trancating jobs

In [96]:
df_analytic.show()

+--------------------+----------+----------+----------+
|           job_title|min_salary|max_salary|avg_salary|
+--------------------+----------+----------+----------+
|3D Computer Visio...|      5409|      5409|    5409.0|
|  Lead Data Engineer|     56000|    276000|  139724.5|
|Head of Machine L...|     79039|     79039|   79039.0|
|     Data Specialist|    165000|    165000|  165000.0|
| Data Analytics Lead|    405000|    405000|  405000.0|
|Machine Learning ...|     12000|    260000|  158412.5|
|   Lead Data Analyst|     19609|    170000|   92203.0|
|Data Engineering ...|     59303|    174000|  123227.2|
|Staff Data Scientist|    105000|    105000|  105000.0|
|       ETL Developer|     54957|     54957|   54957.0|
|Director of Data ...|    113476|    200000|  156738.0|
|Product Data Analyst|      6072|     20000|   13036.0|
|Principal Data Sc...|    148261|    416000|  215242.4|
|        AI Scientist|     12000|    200000|   66135.6|
|Director of Data ...|    130026|    325000|  19

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [97]:
window=Window.orderBy(col("avg_salary").desc())
df_analytic=df_analytic.withColumn("row_id",row_number().over(window))

print all data from df_analytic

In [98]:
df_analytic.show()

+--------------------+----------+----------+----------+------+
|           job_title|min_salary|max_salary|avg_salary|row_id|
+--------------------+----------+----------+----------+------+
| Data Analytics Lead|    405000|    405000|  405000.0|     1|
|Principal Data En...|    185000|    600000|  328333.3|     2|
|Financial Data An...|    100000|    450000|  275000.0|     3|
|Principal Data Sc...|    148261|    416000|  215242.4|     4|
|Director of Data ...|    130026|    325000|  195074.0|     5|
|      Data Architect|     90700|    266400|  177873.9|     6|
|Applied Data Scie...|     54238|    380000|  175655.0|     7|
|  Analytics Engineer|    135000|    205300|  175000.0|     8|
|     Data Specialist|    165000|    165000|  165000.0|     9|
|        Head of Data|     32974|    235000|  160162.6|    10|
|Machine Learning ...|     12000|    260000|  158412.5|    11|
|Data Science Manager|     54094|    241000|  158328.5|    12|
|Director of Data ...|    113476|    200000|  156738.0|

25/09/01 08:30:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [99]:
df_analytic=df_analytic.select("row_id","job_title","min_salary","max_salary","avg_salary")

print df_analytic now

In [100]:
df_analytic.show()

+------+--------------------+----------+----------+----------+
|row_id|           job_title|min_salary|max_salary|avg_salary|
+------+--------------------+----------+----------+----------+
|     1| Data Analytics Lead|    405000|    405000|  405000.0|
|     2|Principal Data En...|    185000|    600000|  328333.3|
|     3|Financial Data An...|    100000|    450000|  275000.0|
|     4|Principal Data Sc...|    148261|    416000|  215242.4|
|     5|Director of Data ...|    130026|    325000|  195074.0|
|     6|      Data Architect|     90700|    266400|  177873.9|
|     7|Applied Data Scie...|     54238|    380000|  175655.0|
|     8|  Analytics Engineer|    135000|    205300|  175000.0|
|     9|     Data Specialist|    165000|    165000|  165000.0|
|    10|        Head of Data|     32974|    235000|  160162.6|
|    11|Machine Learning ...|     12000|    260000|  158412.5|
|    12|Data Science Manager|     54094|    241000|  158328.5|
|    13|Director of Data ...|    113476|    200000|  15

25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 08:30:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [101]:
window1=Window.partitionBy(col("experience_level"))

df_exp_lvl=df.withColumn("biggest_salary",max("salary_in_usd").over(window1))

print here df_exp_lvl

In [102]:
df_exp_lvl.show()

+---+---------+----------------+---------------+--------------------+-------+---------------+-------------+------------------+------------+----------------+------------+--------------+
|_c0|work_year|experience_level|employment_type|           job_title| salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|biggest_salary|
+---+---------+----------------+---------------+--------------------+-------+---------------+-------------+------------------+------------+----------------+------------+--------------+
|  5|     2020|              EN|             FT|        Data Analyst|  72000|            USD|        72000|                US|         100|              US|           L|        250000|
| 10|     2020|              EN|             FT|      Data Scientist|  45000|            EUR|        51321|                FR|           0|              FR|           S|        250000|
| 12|     2020|              EN|             FT|      Data Scientist|  3500

25/09/01 08:30:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///workspace/1_PySpark_Basics/ds_salaries.csv


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [103]:
best=df_exp_lvl.filter("biggest_salary==salary_in_usd")
df_best=df_exp_lvl.select("_c0","job_title","experience_level","employee_residence")\
    .join(other=best.select("experience_level",col("salary").alias("biggest_salary")),on="experience_level",how="left").drop("job_title")

print df_best

In [104]:
df_best.show(df_best.count())

+----------------+---+------------------+--------------+
|experience_level|_c0|employee_residence|biggest_salary|
+----------------+---+------------------+--------------+
|              MI|  0|                DE|        450000|
|              MI|  0|                DE|        450000|
|              SE|  1|                JP|        412000|
|              SE|  2|                GB|        412000|
|              MI|  3|                HN|        450000|
|              MI|  3|                HN|        450000|
|              SE|  4|                US|        412000|
|              EN|  5|                US|        250000|
|              SE|  6|                US|        412000|
|              MI|  7|                HU|        450000|
|              MI|  7|                HU|        450000|
|              MI|  8|                US|        450000|
|              MI|  8|                US|        450000|
|              SE|  9|                NZ|        412000|
|              EN| 10|         

25/09/01 08:30:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, employee_residence
 Schema: _c0, experience_level, employee_residence
Expected: _c0 but found: 
CSV file: file:///workspace/1_PySpark_Basics/ds_salaries.csv


drop duplicates if exist by experience_level

In [105]:
df_best=df_best.dropDuplicates()

print df_best

In [106]:
df_best.show(df_best.count())

25/09/01 08:30:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, employee_residence
 Schema: _c0, experience_level, employee_residence
Expected: _c0 but found: 
CSV file: file:///workspace/1_PySpark_Basics/ds_salaries.csv
25/09/01 08:30:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, employee_residence
 Schema: _c0, experience_level, employee_residence
Expected: _c0 but found: 
CSV file: file:///workspace/1_PySpark_Basics/ds_salaries.csv


+----------------+---+------------------+--------------+
|experience_level|_c0|employee_residence|biggest_salary|
+----------------+---+------------------+--------------+
|              EN| 28|                US|        250000|
|              SE|108|                US|        412000|
|              EN|134|                US|        250000|
|              MI|292|                US|        450000|
|              MI|437|                GR|        450000|
|              EX|187|                FR|        600000|
|              SE|399|                US|        412000|
|              MI|491|                CA|        450000|
|              SE|583|                US|        412000|
|              MI|100|                US|        450000|
|              SE|265|                PR|        412000|
|              SE|402|                US|        412000|
|              SE|552|                US|        412000|
|              MI| 46|                GB|        450000|
|              EN|109|         

create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [107]:


df_new_best=df_best.withColumn("experience_level",\
    when(col("experience_level")=="SE","senior").\
    when(col("experience_level")=="MI","middle").\
    otherwise(None)
)
df_new_best=df_new_best.drop("_c0")

print df_new_best

In [108]:
df_new_best.show()

+----------------+------------------+--------------+
|experience_level|employee_residence|biggest_salary|
+----------------+------------------+--------------+
|            NULL|                US|        250000|
|          senior|                US|        412000|
|            NULL|                US|        250000|
|          middle|                US|        450000|
|          middle|                GR|        450000|
|            NULL|                FR|        600000|
|          senior|                US|        412000|
|          middle|                CA|        450000|
|          senior|                US|        412000|
|          middle|                US|        450000|
|          senior|                PR|        412000|
|          senior|                US|        412000|
|          senior|                US|        412000|
|          middle|                GB|        450000|
|            NULL|                IN|        250000|
|            NULL|                US|        6

25/09/01 08:30:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, employee_residence
 Schema: _c0, experience_level, employee_residence
Expected: _c0 but found: 
CSV file: file:///workspace/1_PySpark_Basics/ds_salaries.csv


write df_new_best like 1.csv and load then it to df_final

In [111]:
df_new_best.write.csv("1.csv",header=True)
df_final=spark.read.csv("1.csv",header=True)

25/09/01 08:31:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, employee_residence
 Schema: _c0, experience_level, employee_residence
Expected: _c0 but found: 
CSV file: file:///workspace/1_PySpark_Basics/ds_salaries.csv


print df_final

In [112]:
df_final.show()

+----------------+------------------+--------------+
|experience_level|employee_residence|biggest_salary|
+----------------+------------------+--------------+
|            NULL|                US|        250000|
|          senior|                US|        412000|
|            NULL|                US|        250000|
|          middle|                US|        450000|
|          middle|                GR|        450000|
|            NULL|                FR|        600000|
|          senior|                US|        412000|
|          middle|                CA|        450000|
|          senior|                US|        412000|
|          middle|                US|        450000|
|          senior|                PR|        412000|
|          senior|                US|        412000|
|          senior|                US|        412000|
|          middle|                GB|        450000|
|            NULL|                IN|        250000|
|            NULL|                US|        6

filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [115]:
df_final=df_final.dropna()

print df_final

In [116]:
df_final.show(df_final.count())

+----------------+------------------+--------------+
|experience_level|employee_residence|biggest_salary|
+----------------+------------------+--------------+
|          senior|                US|        412000|
|          middle|                US|        450000|
|          middle|                GR|        450000|
|          senior|                US|        412000|
|          middle|                CA|        450000|
|          senior|                US|        412000|
|          middle|                US|        450000|
|          senior|                PR|        412000|
|          senior|                US|        412000|
|          senior|                US|        412000|
|          middle|                GB|        450000|
|          senior|                US|        412000|
|          senior|                US|        412000|
|          senior|                US|        412000|
|          middle|                US|        450000|
|          senior|                US|        4

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [119]:
somevar=df_final.select(max(df_final.biggest_salary))
somevar.show()

+-------------------+
|max(biggest_salary)|
+-------------------+
|             450000|
+-------------------+



It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system